In [0]:
import os
import zipfile
import random
import tensorflow as tf
import shutil
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
from os import getcwd
from google.colab import drive

In [0]:
drive.mount('/content/drive/')

myzip= '/content/drive/My Drive/Datasets/Mask-v-NoMask.zip'
zip_ref = zipfile.ZipFile(myzip, 'r')
zip_ref.extractall('/tmp/MaskDataset')
zip_ref.close()

In [0]:
train_mask = os.path.join('/tmp/MaskDataset/Mask-v-NoMask/Train/Mask/')
train_nomask = os.path.join('/tmp/MaskDataset/Mask-v-NoMask/Train/NoMask/')
test_mask = os.path.join('/tmp/MaskDataset/Mask-v-NoMask/Test/Mask/')
test_nomask = os.path.join('/tmp/MaskDataset/Mask-v-NoMask/Test/NoMask/')

print('Training Masked Faces:', len(os.listdir(train_mask)))
print('Training Unmasked Faces:', len(os.listdir(train_nomask)))
print('Testing Masked Faces:', len(os.listdir(test_mask)))
print('Testing Unmasked Faces:', len(os.listdir(test_nomask)))

In [0]:
model = tf.keras.models.Sequential([
    #conv1
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    #conv2
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    #conv3
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    #conv4
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    #conv5
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    #flatten
    tf.keras.layers.Flatten(),
    #hidden
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [0]:
model.compile(optimizer=RMSprop(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [0]:
traindir = "/tmp/MaskDataset/Mask-v-NoMask/Train/"

traindatagen = ImageDataGenerator(rescale=1./255)
traingen = traindatagen.flow_from_directory(traindir, batch_size=64, class_mode='binary', target_size=(150, 150))

validationdir = "/tmp/MaskDataset/Mask-v-NoMask/Test/"
validationdatagen = ImageDataGenerator(rescale=1./255)
validationgen = validationdatagen.flow_from_directory(validationdir, batch_size=25, class_mode='binary', target_size=(150, 150))

In [0]:
history = model.fit(traingen,
                    epochs=100,
                    steps_per_epoch = 32,
                    verbose=1,
                    validation_steps = 10,
                    validation_data=validationgen)

In [0]:
%matplotlib inline

import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc))

plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.figure()

plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")
plt.title('Training and validation loss')

In [0]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(150, 150))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=64)
  print(fn)
  print(classes[0][0])
  if classes[0][0]>0.5:
    print(fn + " is not masked")
  else:
    print(fn + " is masked")

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive 
from google.colab import auth 
from oauth2client.client import GoogleCredentials


In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
model.save('model.h5')
model_file = drive.CreateFile({'title' : 'model.h5'})
model_file.SetContentFile('model.h5')
model_file.Upload()

In [0]:
drive.CreateFile({'id': model_file.get('id')})